In [2]:
###############################################   LAB 5   ##############################################
################### Script written by Dr Alex Ter-Sarkisov@City, University of London, 2021 ############
##################### DEEP LEARNING FOR IMAGE ANALYSIS, MSC IN ARTIFICIAL INTELLIGENCE #################
########################################################################################################
import time
import os, sys, re
from pycocotools.coco import COCO
import torch
import torchvision
import dataset_coco
import numpy as np
import matplotlib.pyplot as plt
import torch.optim as optim
from torch.utils import data

device = torch.device('cpu')
if torch.cuda.is_available():
   device = torch.device('cuda')

print(device)

cuda


In [5]:
###################### load COCO interface, the input is a json file with annotations ####################
coco_interface = COCO("mscoco/annotations/instances_train2017.json")
# all indices of categories in MS COCO:
all_cats = coco_interface.getCatIds()
# add background class
all_cats.insert(0,0)
print(all_cats, len(all_cats))
# get names of cateogories
all_names = coco_interface.loadCats(all_cats[1:])
#print(all_names)
# choose the categories you want to work with
# VERY CAREFUL WITH THIS LIST! SOME CLASSES ARE MISSING, TO TRAIN THE MODEL
# YOU NEED TO ADJUST THE CLASS ID!!!
selected_class_ids = coco_interface.getCatIds(catNms=['toothbrush'])
adjusted_class_ids = {}
for id, cl in enumerate(all_cats):
    adjusted_class_ids[cl] = id
print ("ADJUSTED CLASS IDS:")
print(adjusted_class_ids) 

loading annotations into memory...
Done (t=16.43s)
creating index...
index created!
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 27, 28, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 67, 70, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 84, 85, 86, 87, 88, 89, 90] 81
ADJUSTED CLASS IDS:
{0: 0, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 7: 7, 8: 8, 9: 9, 10: 10, 11: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 27: 25, 28: 26, 31: 27, 32: 28, 33: 29, 34: 30, 35: 31, 36: 32, 37: 33, 38: 34, 39: 35, 40: 36, 41: 37, 42: 38, 43: 39, 44: 40, 46: 41, 47: 42, 48: 43, 49: 44, 50: 45, 51: 46, 52: 47, 53: 48, 54: 49, 55: 50, 56: 51, 57: 52, 58: 53, 59: 54, 60: 55, 61: 56, 62: 57, 63: 58, 64: 59, 65: 60, 67: 61, 70: 62, 72: 63, 73: 64, 74: 65, 75: 66, 76: 67, 77: 68, 78: 69, 79: 70, 80: 71, 81: 72, 82: 73, 84: 7

In [7]:
####################################################################
# load ids of images with this class
# Dataset class takes this list as an input and creates data objects 
im_ids = coco_interface.getImgIds(catIds=selected_class_ids)
####################################################################
# selected class ids: extract class id from the annotation
coco_data_args = {'datalist':im_ids, 'coco_interface':coco_interface, 'coco_classes_idx':selected_class_ids,'stage':'train', 'adjusted_classes_idx':adjusted_class_ids}
coco_data = dataset_coco.COCOData(**coco_data_args)
coco_dataloader_args = {'batch_size':1, 'shuffle':True}
coco_dataloader = data.DataLoader(coco_data, **coco_dataloader_args)
#####################################################################

In [23]:
################### MASK R-CNN MODEL ################################################
maskrcnn_args = {'num_classes':81, 'min_size':512, 'max_size':800}
maskrcnn_model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=False,**maskrcnn_args)
print(maskrcnn_model)
pretrained_weights = torch.load(os.path.join('mscoco', 'maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth'))
# # copy only backbone weights
for _n, _par in maskrcnn_model.state_dict().items():
     if 'backbone' in _n:
        print(_n)
        _par.requires_grad = False
        _par.copy_(pretrained_weights[_n])
        _par.requires_grad = True
     else:
        print(_n, "not a backbone")

if device == torch.device('cuda'):
   maskrcnn_model = maskrcnn_model.to(device)

maskrcnn_model.train()

maskrcnn_optimizer_pars = {'lr':1e-5}
maskrcnn_optimizer = optim.Adam(list(maskrcnn_model.parameters()),**maskrcnn_optimizer_pars)

total_epochs = 5

start_time = time.time()


MaskRCNN(
  (transform): GeneralizedRCNNTransform()
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d()
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d()
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d()
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d()
          (relu): ReLU(inplace=True)
          (downsample): Sequential(
            (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (1): FrozenBatchNorm2d()
          )
    

In [24]:
start_time = time.time()

for e in range(total_epochs):
    epoch_loss = 0    
    for id, b in enumerate(coco_dataloader):
        maskrcnn_optimizer.zero_grad()
        X,y = b
        if device==torch.device('cuda'):
            X, y['labels'], y['boxes'], y['masks'] = X.to(device), y['labels'].to(device), y['boxes'].to(device), y['masks'].to(device)
        images = [im for im in X]
        targets = []
        lab={}
        # THIS IS IMPORTANT!!!!!
        # get rid of the first dimension (batch)
        # IF you have >1 images, make another loop
        # REPEAT: DO NOT USE BATCH DIMENSION 
        # Pytorch is sensitive to formats. Labels must be int64, bboxes float32, masks uint8
        lab['boxes'] = y['boxes'].squeeze_(0)
        lab['labels'] = y['labels'].squeeze_(0)
        lab['masks'] = y['masks'].squeeze_(0)
        targets.append(lab)
        # avoid empty objects
        if len(targets)>0:
           loss = maskrcnn_model(images, targets)
           total_loss = 0
           for k in loss.keys():
               total_loss += loss[k]
           epoch_loss += total_loss.item()
           total_loss.backward()        
           maskrcnn_optimizer.step()
    epoch_loss = epoch_loss/len(coco_dataloader)
    print("Loss in epoch {0:d} = {1:.3f}".format(e, epoch_loss))



Loss in epoch 0 = 0.878
Loss in epoch 1 = 0.607
Loss in epoch 2 = 0.526
Loss in epoch 3 = 0.477
Loss in epoch 4 = 0.434


In [26]:
maskrcnn_model.eval()
torch.save(maskrcnn_model, "maskrcnn_mscoco.pth")